In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 14.1 MB/s 
     |████████████████████████████████| 2.9 MB 49.0 MB/s 
     |████████████████████████████████| 880 kB 49.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2ffa0df609a5cf9b1bc23ef34f81196749c8ae843bc10db3be39bbff70f7a54c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [4]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [5]:
! pip install torchvision==0.7.0

     |████████████████████████████████| 5.9 MB 13.1 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113


In [6]:
! pip install numpy==1.17.0

     |████████████████████████████████| 20.3 MB 10.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.17.0 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.17.0 which is incompatible.
pywavelets 1.3.0 requires numpy>=1.17.3, but you have numpy 1.17.0 which is incompatible.
pandas 1.3.5 requires numpy>=1.17.3; platform_machine != "aarch64" and platform_machine != "arm64" and python_version < "3.10", but you have numpy 1.17.0 which is incomp

## logging
Transformers 有一个集中的日志记录系统，因此您可以轻松设置库的详细程度。
目前该库的默认详细程度是WARNING.
* transformers.logging.CRITICALor transformers.logging.FATAL(int value, 50)：只报告最严重的错误。
* transformers.logging.ERROR(int value, 40)：只报告错误。
* transformers.logging.WARNINGor transformers.logging.WARN(int value, 30)：只报告错误和警告。这是库使用的默认级别。
* transformers.logging.INFO(int value, 20)：报告错误、警告和基本信息。
* transformers.logging.DEBUG(int value, 10)：上报所有信息。

In [7]:
import random
import json
import pandas as pd
import numpy as np
import os
import torch
from transformers import logging
from transformers import BertTokenizer
from collections import defaultdict
from tqdm import tqdm
def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  return seed

logging.set_verbosity_info()
seed_everything(2022)

2022

In [8]:
corpus_path = '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM'
model_path = '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model'
output_dir = '.'

In [9]:
tokenizer = BertTokenizer.from_pretrained(model_path)

Model name '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model' is a path, a model identifier, or url to a directory containing tokenizer files.
Didn't find file /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model/added_tokens.json. We won't load it.
Didn't find file /content/drive/MyDrive/Colab Notebooks/datase

In [10]:
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [11]:
def read_data(config, tokenizer):
  train_df = parse_data(os.path.join(corpus_path, 'train.json'), data_type = 'train')
  dev_df = parse_data(os.path.join(corpus_path, 'dev.json'), data_type = 'dev')
  test_df = parse_data(os.path.join(corpus_path, 'test.json'), data_type = 'test')

  train_df.append(dev_df)
  train_df.append(test_df)
  inputs = defaultdict(list)


  for i, row in tqdm(train_df.iterrows(), desc= f'Preprocessing train data', total = len(train_df)):
      inputs_dict = tokenizer.encode_plus(row[0] + row[1], add_special_tokens = True,
                                          return_token_type_ids = True, return_attention_mask = True)
      inputs['input_ids'].append(inputs_dict['input_ids'])
      inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
      inputs['attention_mask'].append(inputs_dict['attention_mask'])
    
    
    
  return inputs

In [12]:
data = read_data(corpus_path, tokenizer)

Preprocessing train data: 100%|██████████| 34334/34334 [00:32<00:00, 1054.99it/s]


In [13]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict
  
  # 返回一个example
  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [14]:
train_dataset = AFQMCDataset(data)

```
from transformers import BertTokenizerFast, BertForMaskedLM
from transformers import LineByLineTextDataset, Trainer, TrainingArguments, DataCollatorForLanguageModeling
def train(self):
    # model_path是预训练模型的位置
    # 分词 BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained(model_path, max_len=512)
    # 定义一个模型 BertForMaskedLM
    model = BertForMaskedLM.from_pretrained(model_path)
    # daseset LineByLineTextDataset
    dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path="语料文件.txt",
        block_size=128,
    )
    # collator DataCollatorForLanguageModeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.10
    )
    # TrainingArguments 训练参数，使用的接口类
    training_args = TrainingArguments(
        output_dir="xxxx",
        ...各种参数，参见文档或者源码
    )
    
    # 模型训练用Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model("保存位置")
```

In [15]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained(model_path)

loading configuration file /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

loading weights file /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model/pytorch_model.bin
Some weights of the model checkpoint at /content/drive/My

#### NOTE1

In [16]:
# N -Gram N: 1,2,3
ngrams = np.arange(1, 4, dtype = np.int64)
print('ngrams:',ngrams)
pvals = 1. / np.arange(1, 4)
print('pvals(1):',pvals) 
p = pvals.sum(keepdims = True)
print('psum:',p)
pvals /= pvals.sum(keepdims = True)
print('pvals(2):',pvals)
pvals = pvals[::-1]
print('pvals(3):',pvals)

ngrams: [1 2 3]
pvals(1): [1.         0.5        0.33333333]
psum: [1.83333333]
pvals(2): [0.54545455 0.27272727 0.18181818]
pvals(3): [0.18181818 0.27272727 0.54545455]


#### NOTE2

In [17]:
# 非special token的位置
cand_indexes = [[1],[2],[3],[4],[5],[7],[8],[9]]
# ngrams = [1,2,3]
ngram_indexes = []
for idx in range(len(cand_indexes)):
  ngram_index = []
  for n in ngrams:
    # 生成1/2/3-gram
    ngram_index.append(cand_indexes[idx:idx + n])
    print('ngram_index:',ngram_index)
  ngram_indexes.append(ngram_index)

print('ngram_indexes:',ngram_indexes)

ngram_index: [[[1]]]
ngram_index: [[[1]], [[1], [2]]]
ngram_index: [[[1]], [[1], [2]], [[1], [2], [3]]]
ngram_index: [[[2]]]
ngram_index: [[[2]], [[2], [3]]]
ngram_index: [[[2]], [[2], [3]], [[2], [3], [4]]]
ngram_index: [[[3]]]
ngram_index: [[[3]], [[3], [4]]]
ngram_index: [[[3]], [[3], [4]], [[3], [4], [5]]]
ngram_index: [[[4]]]
ngram_index: [[[4]], [[4], [5]]]
ngram_index: [[[4]], [[4], [5]], [[4], [5], [7]]]
ngram_index: [[[5]]]
ngram_index: [[[5]], [[5], [7]]]
ngram_index: [[[5]], [[5], [7]], [[5], [7], [8]]]
ngram_index: [[[7]]]
ngram_index: [[[7]], [[7], [8]]]
ngram_index: [[[7]], [[7], [8]], [[7], [8], [9]]]
ngram_index: [[[8]]]
ngram_index: [[[8]], [[8], [9]]]
ngram_index: [[[8]], [[8], [9]], [[8], [9]]]
ngram_index: [[[9]]]
ngram_index: [[[9]], [[9]]]
ngram_index: [[[9]], [[9]], [[9]]]
ngram_indexes: [[[[1]], [[1], [2]], [[1], [2], [3]]], [[[2]], [[2], [3]], [[2], [3], [4]]], [[[3]], [[3], [4]], [[3], [4], [5]]], [[[4]], [[4], [5]], [[4], [5], [7]]], [[[5]], [[5], [7]], [[5],

In [19]:
for cand_index_set in ngram_indexes:
  # n 是n-gram的n
  n = np.random.choice(ngrams[:len(cand_index_set)],
                 p = pvals[:len(cand_index_set)] / pvals[:len(cand_index_set)].sum(keepdims = True))
  print('n:',n)
  print('cand_index_set:',cand_index_set[n - 1])
  # 合并列表
  index_set = sum(cand_index_set[n - 1], [])
  print('index_set:',index_set)
  break

n: 3
cand_index_set: [[1], [2], [3]]
index_set: [1, 2, 3]
